In [96]:
# import libraries
import pandas as pd
import numpy as np
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [97]:
# load data
ed = pd.read_csv("train.csv")
metadata = pd.read_csv("metadata_TdPVeJC.csv")
tdf = pd.read_csv("test_monthly_naturalized_flow.csv")
df = pd.read_csv("train_monthly_naturalized_flow.csv")
ss = pd.read_csv("submission_format.csv")

In [98]:
metadata['n_months'] = metadata.season_end_month-metadata.season_start_month

In [99]:
ss

,site_id,issue_date,volume_10,volume_50,volume_90
0,hungry_horse_reservoir_inflow,2005-01-01,0.0,0.0,0.0
1,hungry_horse_reservoir_inflow,2005-01-08,0.0,0.0,0.0
2,hungry_horse_reservoir_inflow,2005-01-15,0.0,0.0,0.0
3,hungry_horse_reservoir_inflow,2005-01-22,0.0,0.0,0.0
4,hungry_horse_reservoir_inflow,2005-02-01,0.0,0.0,0.0
...,...,...,...,...,...
7235,owyhee_r_bl_owyhee_dam,2023-06-22,0.0,0.0,0.0
7236,owyhee_r_bl_owyhee_dam,2023-07-01,0.0,0.0,0.0
7237,owyhee_r_bl_owyhee_dam,2023-07-08,0.0,0.0,0.0
7238,owyhee_r_bl_owyhee_dam,2023-07-15,0.0,0.0,0.0


In [100]:
ss[~ss.site_id.isin(df.site_id)]["site_id"].unique()

array(['san_joaquin_river_millerton_reservoir',
       'merced_river_yosemite_at_pohono_bridge',
       'american_river_folsom_lake'], dtype=object)

In [101]:
ed=ed.dropna()
ed

,site_id,year,volume
0,hungry_horse_reservoir_inflow,1911,2178.526
1,hungry_horse_reservoir_inflow,1912,1973.732
2,hungry_horse_reservoir_inflow,1913,3220.839
12,hungry_horse_reservoir_inflow,1923,1825.846
13,hungry_horse_reservoir_inflow,1924,1743.380
...,...,...,...
2172,owyhee_r_bl_owyhee_dam,2014,69.818
2173,owyhee_r_bl_owyhee_dam,2016,213.150
2174,owyhee_r_bl_owyhee_dam,2018,118.743
2175,owyhee_r_bl_owyhee_dam,2020,180.786


In [102]:
df=pd.merge(df,metadata,on="site_id",how="left")
tdf=pd.merge(tdf,metadata,on="site_id",how="left")
df = pd.merge(df,ed.rename({"volume":"target","year":"forecast_year"},axis=1),on=["site_id","forecast_year"],how="left")


In [103]:
target = "target"

In [104]:
# define preprocess function
def pp(df):
    df["v1"] = df.groupby(["site","year"])["volume"].rolling(100,min_periods=1).sum().values
    df["v2"] = df["volume"].values
    df["v2"][df["month"]<df["season_start_month"]] = np.nan
    df["v2"][df["month"]>df["season_end_month"]] = np.nan
    df["v2"] = df.groupby(["site","year"])["v2"].rolling(100,min_periods=1).sum().values
    return(df)

In [105]:
ss["date"] = pd.to_datetime(ss["issue_date"])
ss["year"] = ss["date"].dt.year
ss["month"] = ss["date"].dt.month
ss

,site_id,issue_date,volume_10,volume_50,volume_90,date,year,month
0,hungry_horse_reservoir_inflow,2005-01-01,0.0,0.0,0.0,2005-01-01,2005,1
1,hungry_horse_reservoir_inflow,2005-01-08,0.0,0.0,0.0,2005-01-08,2005,1
2,hungry_horse_reservoir_inflow,2005-01-15,0.0,0.0,0.0,2005-01-15,2005,1
3,hungry_horse_reservoir_inflow,2005-01-22,0.0,0.0,0.0,2005-01-22,2005,1
4,hungry_horse_reservoir_inflow,2005-02-01,0.0,0.0,0.0,2005-02-01,2005,2
...,...,...,...,...,...,...,...,...
7235,owyhee_r_bl_owyhee_dam,2023-06-22,0.0,0.0,0.0,2023-06-22,2023,6
7236,owyhee_r_bl_owyhee_dam,2023-07-01,0.0,0.0,0.0,2023-07-01,2023,7
7237,owyhee_r_bl_owyhee_dam,2023-07-08,0.0,0.0,0.0,2023-07-08,2023,7
7238,owyhee_r_bl_owyhee_dam,2023-07-15,0.0,0.0,0.0,2023-07-15,2023,7


In [106]:
df["site"] = le.fit_transform(df["site_id"]).astype("int")
tdf["site"] = le.transform(tdf["site_id"]).astype("int")


In [107]:
df = df.sort_values(["site","year","month"])
tdf = tdf.sort_values(["site","year","month"])

In [108]:
cols = ['forecast_year', 'year', 'month', 'volume', 'season_start_month',
       'season_end_month', 'elevation', 'latitude', 'longitude',
       'drainage_area', 'usgs_id', 'site','n_months'] 

In [109]:
na_val = np.nan

In [110]:
df=df[df[target].notna()]

In [111]:
# preprocess and train quantile regression

X_train = pp(df[cols]).fillna(na_val)
X_test = pp(tdf[cols]).fillna(na_val)
for i in [10,50,90]:
    model = LGBMRegressor(n_estimators=80,objective="quantile",alpha=(i/100))

    model.fit(X_train,df[target])
    tdf[f"volume_{i}"] = model.predict(X_test)
tdf


C:\Users\hp\AppData\Local\Temp\ipykernel_18220\1498199745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["v1"] = df.groupby(["site","year"])["volume"].rolling(100,min_periods=1).sum().values
C:\Users\hp\AppData\Local\Temp\ipykernel_18220\1498199745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["v2"] = df["volume"].values
C:\Users\hp\AppData\Local\Temp\ipykernel_18220\1498199745.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

,site_id,forecast_year,year,month,volume,season_start_month,season_end_month,elevation,latitude,longitude,...,nrcs_id,nrcs_name,rfc_id,rfc_name,rfc,n_months,site,volume_10,volume_50,volume_90
450,animas_r_at_durango,2005,2004,10,31.855,4,7,6502.0,37.27917,-107.88034,...,09361500:CO:USGS,Animas R at Durango,DRGC2,Animas - Durango,CBRFC,3,0,268.814181,343.464813,447.374274
451,animas_r_at_durango,2005,2004,11,26.100,4,7,6502.0,37.27917,-107.88034,...,09361500:CO:USGS,Animas R at Durango,DRGC2,Animas - Durango,CBRFC,3,0,275.617367,324.204803,445.054782
452,animas_r_at_durango,2005,2004,12,19.503,4,7,6502.0,37.27917,-107.88034,...,09361500:CO:USGS,Animas R at Durango,DRGC2,Animas - Durango,CBRFC,3,0,252.668736,324.204803,441.236672
453,animas_r_at_durango,2005,2005,1,19.557,4,7,6502.0,37.27917,-107.88034,...,09361500:CO:USGS,Animas R at Durango,DRGC2,Animas - Durango,CBRFC,3,0,249.835387,333.710764,438.526874
454,animas_r_at_durango,2005,2005,2,20.755,4,7,6502.0,37.27917,-107.88034,...,09361500:CO:USGS,Animas R at Durango,DRGC2,Animas - Durango,CBRFC,3,0,273.316087,333.710764,435.548056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,yampa_r_nr_maybell,2023,2023,2,12.267,4,7,5900.0,40.50275,-108.03342,...,09251000:CO:USGS,Yampa R nr Maybell,MBLC2,Yampa - Maybell Nr,CBRFC,3,22,438.061589,716.378795,992.095564
626,yampa_r_nr_maybell,2023,2023,3,17.226,4,7,5900.0,40.50275,-108.03342,...,09251000:CO:USGS,Yampa R nr Maybell,MBLC2,Yampa - Maybell Nr,CBRFC,3,22,432.816116,780.942527,992.095564
627,yampa_r_nr_maybell,2023,2023,4,224.967,4,7,5900.0,40.50275,-108.03342,...,09251000:CO:USGS,Yampa R nr Maybell,MBLC2,Yampa - Maybell Nr,CBRFC,3,22,663.708304,994.873578,1281.950569
628,yampa_r_nr_maybell,2023,2023,5,739.692,4,7,5900.0,40.50275,-108.03342,...,09251000:CO:USGS,Yampa R nr Maybell,MBLC2,Yampa - Maybell Nr,CBRFC,3,22,1550.593789,1296.757012,1473.734483


In [112]:
pred = pd.merge(ss[["site_id","issue_date","date","year","month"]], tdf[["site_id","year","month","volume_10","volume_50","volume_90"]], on=["site_id","year","month"], how="left")

In [113]:
# post -process predictions
for i in [10,50,90]:
    pred[f"volume_{i}"] = pred.groupby(["site_id","year"])[f"volume_{i}"].fillna(method="ffill")
    pred[f"volume_{i}"] = pred[f"volume_{i}"].fillna(pred[f"volume_{i}"].mean())

In [114]:
# save predictions
sub=pred.drop(["date","year","month"],axis=1)
sub.to_csv("submission.csv",index=None)
sub

,site_id,issue_date,volume_10,volume_50,volume_90
0,hungry_horse_reservoir_inflow,2005-01-01,1537.153119,2281.339469,2395.756664
1,hungry_horse_reservoir_inflow,2005-01-08,1537.153119,2281.339469,2395.756664
2,hungry_horse_reservoir_inflow,2005-01-15,1537.153119,2281.339469,2395.756664
3,hungry_horse_reservoir_inflow,2005-01-22,1537.153119,2281.339469,2395.756664
4,hungry_horse_reservoir_inflow,2005-02-01,1505.309354,2002.778999,2553.533256
...,...,...,...,...,...
7235,owyhee_r_bl_owyhee_dam,2023-06-22,332.555944,339.803923,450.977326
7236,owyhee_r_bl_owyhee_dam,2023-07-01,332.555944,339.803923,450.977326
7237,owyhee_r_bl_owyhee_dam,2023-07-08,332.555944,339.803923,450.977326
7238,owyhee_r_bl_owyhee_dam,2023-07-15,332.555944,339.803923,450.977326
